In [2]:
import numpy as np
from simulation_framework.item_response_simulation import item_response_simulation
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma
import scipy

In [2]:
def params_to_vector(model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension
    A_flat = model.item_parameters["discrimination_matrix"].flatten()
    A_flat = A_flat[A_flat != 0]
    A_cut = len(A_flat)
    delta_cut = A_cut+item_dimension
    delta = model.item_parameters["intercept_vector"]
    sigma_flat = model.person_parameters["covariance"][np.triu_indices(
            model.latent_dimension, k=1)]
    return(np.concatenate((A_flat, delta, sigma_flat), axis=0))

def vector_to_params(vector, model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension 
    q_matrix = model.item_parameters["q_matrix"] 
    q_flat = q_matrix.flatten()
    A_cut = len(q_flat[q_flat != 0])
    delta_cut = A_cut+item_dimension
    A = vector[0:A_cut]
    A = model.fill_zero_discriminations(A).reshape((item_dimension, latent_dimension))
    delta = vector[A_cut:delta_cut]
    item_parameters = {"discrimination_matrix": A, "intercept_vector": delta}
    corr = vector[delta_cut: len(vector)]
    sigma = model.corr_to_sigma(corr, False)
    return({"item_parameters": item_parameters, "person_parameters": {"covariance": sigma}})

def direct_marginal_optimization(model, response_data):
    #Get initial parameters
    x0 = params_to_vector(model)
    response_data = response_data.to_numpy()
    #Optimize with GA
    def marginal_l_func(input_vector):
        parameters = vector_to_params(input_vector, model)
        try:
            model.set_parameters(parameters)
        except Exception:
            return(np.inf)
        result = -1*model.marginal_response_loglikelihood(response_data)
        return(result)
    es = cma.CMAEvolutionStrategy(x0=x0, sigma0=0.5)
    es.optimize(marginal_l_func, maxfun=100000)
    #Get result
    result = es.result.xfavorite
    return(vector_to_params(result, model))

In [3]:
from models.mirt_2pl import mirt_2pl

def rmse(y_pred: np.array, y_true: np.array) -> float:
    MSE = np.square(np.subtract(y_pred.flatten(), y_true.flatten())).mean()
    RMSE = np.sqrt(MSE)
    return(float(RMSE))

def experiment_performance(estimated_parameter_dict, real_parameter_dict):
    res_dict = {}
    if "item_parameters" in estimated_parameter_dict.keys():
        A_pred = estimated_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_pred = estimated_parameter_dict["item_parameters"]["intercept_vector"]

        A_true = real_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_true = real_parameter_dict["item_parameters"]["intercept_vector"]

        print("Absolute diff in A:")
        print(np.abs(A_true-A_pred))

        print("Absolute diff in delta:")
        print(np.abs(delta_true-delta_pred))

        res_dict["rmse_A"] = rmse(A_pred, A_true)
        res_dict["rmse_delta"] = rmse(delta_true, delta_pred)

    if "person_parameters" in estimated_parameter_dict.keys():
        sigma_pred = estimated_parameter_dict["person_parameters"]["covariance"]
        sigma_true = real_parameter_dict["person_parameters"]["covariance"]

        print("Absolute diff in sigma:")
        print(np.abs(sigma_true-sigma_pred)) # TODO: Evtl. die Hauptdiagonale ausschließen!

        res_dict["rmse_sigma"] = rmse(sigma_true, sigma_pred) 
    if len(res_dict.keys())==0:
        raise Exception("No performance to calculate")
    return(res_dict)

def standardize_parameters(parameter_dict):
        covariance = parameter_dict["person_parameters"]["covariance"]
        A = parameter_dict["item_parameters"]["discrimination_matrix"]
        if len(covariance.shape)<=1:
            correlation_matrix  = np.array([[1]])
            inv_sqrt_cov = np.array([[np.sqrt(1/covariance)]])
            A = np.multiply(A, inv_sqrt_cov)
        else:
            sd_vector = np.sqrt(covariance.diagonal())
            inv_sd_matrix = np.linalg.inv(np.diag(sd_vector))
            correlation_matrix = np.dot(
            np.dot(inv_sd_matrix, covariance), inv_sd_matrix)
            A = np.dot(A, np.linalg.inv(inv_sd_matrix))
        parameter_dict["item_parameters"]["discrimination_matrix"] = A
        parameter_dict["person_parameters"]["covariance"] = correlation_matrix
        return(parameter_dict)

def create_parameter_dict(estimated_early_parameters, real_early_parameters, estimated_late_parameters, real_late_parameters):
    parameter_dict = {"real_early_parameters": real_early_parameters,
                   "estimated_early_parameters": estimated_early_parameters}
    return(parameter_dict)


def create_performance_dict(parameter_dict, run_dict, sample=None, baselines=None, model=None):
    result_dict = parameter_dict
    result_dict["sample"] = sample
    #Model Performance
    result_dict["early_performance"] = {}
    result_dict["early_performance"]["rmse"] = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=result_dict["estimated_early_parameters"])
    #Baseline's Performance
    baselines["early_initial"]["performance"] = {"rmse": experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=baselines["early_initial"]["parameters"])}
    if "girth" in baselines.keys():
        girth_data = result_dict["sample"]["early_responses"].to_numpy().transpose()
        if model.latent_dimension == 1:
            girth_estimates = twopl_mml(girth_data)
        else:
            girth_estimates = multidimensional_twopl_mml(girth_data, n_factors=model.latent_dimension)
        girth_item_parameters = {"discrimination_matrix": girth_estimates["Discrimination"], "intercept_vector": girth_estimates["Difficulty"]}
        girth_parameters = {"item_parameters": girth_item_parameters}
        girth_estimated_covariance = np.cov(girth_estimates["Ability"], rowvar=True)
        girth_parameters["person_parameters"] = {"covariance": girth_estimated_covariance}
        girth_parameters = standardize_parameters(girth_parameters)
        girth_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=girth_parameters)
        baselines["girth"]["parameters"] = girth_parameters
        baselines["girth"]["performance"] = {"rmse": girth_performance_rmse}

    if "early_direct" in baselines.keys():
        dir_model = mirt_2pl(item_dimension=model.item_dimension, latent_dimension=model.latent_dimension, Q=model.item_parameters["q_matrix"])
        direct_early_item_parameters = direct_marginal_optimization(dir_model, response_data=sample["early_responses"])
        direct_early_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=direct_early_item_parameters)
        baselines["early_direct"]["parameters"] = direct_early_item_parameters
        baselines["early_direct"]["performance"] = {"rmse": direct_early_performance_rmse}

    result_dict["baselines"] = baselines
    likelihood = calculate_marginal_likelihoods(model=model, response_data=sample["early_responses"], real_parameters=result_dict["real_early_parameters"],
                                                initial_parameters=baselines["early_initial"]["parameters"], estimated_parameters=result_dict["estimated_early_parameters"])
    result_dict["early_performance"]["marginal_likelihood"] = likelihood
    result_dict["early_performance"]["run"] = run_dict["early"]
    return(result_dict)

def calculate_marginal_likelihoods(model, response_data, real_parameters, initial_parameters, estimated_parameters):
    model.set_parameters(initial_parameters)
    initial_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(real_parameters)
    optimal_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(estimated_parameters)
    marginal_likelihood_estimated = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    likelihood_dict = {"optimal": optimal_marginal_likelihood, "estimated": marginal_likelihood_estimated, "initial": initial_marginal_likelihood}
    return(likelihood_dict)

In [4]:
def direct_optimization_benchmark():
    latent_dimension = 2
    item_dimension = 6
    sample_size=100

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(sample_size)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])

    estimated_params = direct_marginal_optimization(model, response_data=sample["early_responses"])
    return(estimated_params)


## Experiment 0: MIRT-2PL Performance Benchmark

In [5]:
def mirt_performance_benchmark() -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=100

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(sample_size)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}, "early_direct": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)
    return(performance_dict)

In [6]:

def print_result(result_dict, description=""):
    #Description
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    #Experiment Properties:
    latent_dimension = result_dict["sample"]["latent_dimension"]
    item_dimension = result_dict["sample"]["item_dimension"]
    sample_size = result_dict["sample"]["sample_size"]
    print("Latent dimension: {0},  Item dimension: {1}, sample size {2} \\".format(latent_dimension, item_dimension, sample_size))
    #Performance/time
    ep_dict = result_dict["early_performance"]
    runtime = np.round(ep_dict["run"]["runtime"], 2)
    steps = ep_dict["run"]["number_steps"]
    print("Runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(runtime, steps, np.round(runtime/steps, 2)))
    #Performance/results
    l_optimal = np.round(ep_dict["marginal_likelihood"]["optimal"], 2)
    l_estimated = np.round(ep_dict["marginal_likelihood"]["estimated"], 2)
    l_real = np.round(ep_dict["marginal_likelihood"]["initial"], 2)
    print("Optimal marginal Likelihood: {0}, Estimated: {1}, Initial {2}".format(l_optimal,l_estimated,l_real))
    #print("Performance: rmse-mean = {0} \\".format(np.round(np.mean(np.array(list(result_dict["early_performance"].values()))), 4)))
    rmse_model = ep_dict["rmse"]
    
    rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    rmse_frame = rmse_frame.append(rmse_model, ignore_index=True)
    index = ["estimated"]
    for baseline in list(result_dict["baselines"].keys()):
        rmse_baseline = result_dict["baselines"][baseline]["performance"]["rmse"]
        rmse_frame = rmse_frame.append(rmse_baseline, ignore_index=True)
        index.append(baseline)
    rmse_frame.index = index
    print(rmse_frame.to_markdown())
                                             

In [7]:
performance_dict = mirt_performance_benchmark()

EM Iteration 2
Current Monte Carlo Sample size: 300
[[1.         0.50955533]
 [0.50955533 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 1.5345205134548143, current marginal loglikelihood: -373.66879312920685
EM Iteration 3
Current Monte Carlo Sample size: 300
[[1.         0.51132214]
 [0.51132214 1.        ]]
Step: 3: current parameter_diff: 1.1496593361417635, current marginal loglikelihood: -371.8482760165528
EM Iteration 4
Current Monte Carlo Sample size: 330
[[1.         0.49955822]
 [0.49955822 1.        ]]
Step: 4: current parameter_diff: 1.0378484654568882, current marginal loglikelihood: -370.6098519099325
EM Iteration 5
Current Monte Carlo Sample size: 363
[[1.         0.50319628]
 [0.50319628 1.        ]]
Step: 5: current parameter_diff: 0.8044879101413374, current marginal loglikelihood: -369.27034978664943
EM Iteration 6
Current Monte Carlo Sample size: 399
[[1.         0.50370618]
 [0.50370618 1.        ]]
Step: 6: current parameter_diff: 0.8104581755325072, current marginal loglikelihood: -369.1616783778356
EM Iteration 7
Current Monte Carlo Sample size: 438
[[1.         0

c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [3, 6, 9] are not finite but [inf, inf, inf]. (class=CMAEvolutionStrategy method=ask iteration=4)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [10] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=6)
  warnings.warn(msg + ' (' +


[[1.         1.13184141]
 [1.13184141 1.        ]]
[[1.         1.14029677]
 [1.14029677 1.        ]]
[[1.         1.03185824]
 [1.03185824 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [10] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=15)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [2, 9] are not finite but [inf, inf]. (class=CMAEvolutionStrategy method=ask iteration=18)
  warnings.warn(msg + ' (' +


[[1.         1.09200055]
 [1.09200055 1.        ]]
[[1.         1.14279112]
 [1.14279112 1.        ]]
[[1.        1.0559787]
 [1.0559787 1.       ]]
[[1.         1.02270144]
 [1.02270144 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [5] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=26)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [1, 5] are not finite but [inf, inf]. (class=CMAEvolutionStrategy method=ask iteration=27)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [6] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=28)
  warnings.warn(msg + ' (' +


   56    672 3.669582785863784e+02 2.4e+00 6.10e-02  4e-02  7e-02 0:03.2
  100   1200 3.666234875412319e+02 3.2e+00 3.94e-02  2e-02  5e-02 0:05.8
  186   2232 3.664839823266960e+02 5.7e+00 2.99e-02  1e-02  4e-02 0:10.9
  200   2400 3.666607845292899e+02 6.3e+00 1.72e-02  8e-03  2e-02 0:11.7
  300   3600 3.663931043568954e+02 1.0e+01 1.04e-02  5e-03  1e-02 0:17.5
  400   4800 3.663900371537832e+02 1.8e+01 2.70e-02  1e-02  4e-02 0:23.8
  500   6000 3.663403986926041e+02 1.8e+01 2.62e-02  9e-03  3e-02 0:30.4
  600   7200 3.664181422211946e+02 2.3e+01 1.82e-02  6e-03  2e-02 0:37.2
  700   8400 3.664285330482364e+02 3.7e+01 9.72e-03  3e-03  1e-02 0:43.7
  800   9600 3.661100435690640e+02 6.3e+01 2.38e-02  7e-03  3e-02 0:50.2
  900  10800 3.663329541166315e+02 6.7e+01 1.10e-02  2e-03  1e-02 0:56.7
  980  11760 3.662611574835303e+02 8.8e+01 7.31e-03  2e-03  8e-03 1:02.1
Absolute diff in A:
[[0.26219961 0.        ]
 [0.12162323 0.        ]
 [0.         0.12362127]
 [0.18017018 0.41543923]
 [1.

In [8]:
print_result(performance_dict, "Increasing Monte Carlo sample-size based on ttest with normal MC")

------------------------------------
##### Results for Increasing Monte Carlo sample-size based on ttest with normal MC
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 24.05 seconds, 52 steps, 0.46 seconds per step \
Optimal marginal Likelihood: -370.02, Estimated: -366.57, Initial -377.33
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.651954 |     0.386172 |    0.101894  |
| early_initial | 0.509902 |     0.356989 |    0.212132  |
| girth         | 1.22228  |     0.909143 |    0.0379023 |
| early_direct  | 0.607256 |     0.332714 |    0.0745539 |


------------------------------------
##### Results for Naive, all-Genetic, no vectorization
Runtime: 29100 seconds, 100+ steps, 291 seconds per step\
Performance: rmse-mean = 0.5896464054719196

------------------------------------
##### Results for Vectorized q_item
Runtime: 613.3871161937714 seconds, 100+ steps, 6.0731397642947655 seconds per step \
Performance: rmse-mean = 0.6466789259215839

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2239.073846578598 seconds, 100+ steps, 22.169047985926714 seconds per step \
Performance: rmse-mean = 0.6384660709857508

------------------------------------
##### Results for Debug GA sorting, Add Likelihood-based stopping criterion
Runtime: 135.11203932762146 seconds, 7 steps, 19.301719903945923 seconds per step \
Performance: rmse-mean = 0.5616448369465917

------------------------------------
##### Results for Add vectorization in Q_0
Runtime: 228.78 seconds, 17 steps, 13.46 seconds per step \
Performance: rmse-mean = 0.6174

------------------------------------
##### Results for Add vectorization in E-Step normalizing constant
Runtime: 28.37 seconds, 9 steps, 3.15 seconds per step \
Performance: rmse-mean = 0.5808

------------------------------------
##### Results for Add Suggested Initial Parameters (Zhang)
Runtime: 61.22 seconds, 19 steps, 3.22 seconds per step \
Performance: rmse-mean = 0.5625

------------------------------------
##### Results for Fixed Q_item
Runtime: 81.11 seconds, 12 steps, 6.76 seconds per step \
Optimal marginal Likelihood: -375.43, Estimated: -393.8, Initial -391.83
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.762005 |     0.512703 |     0.600404 |
| Girth     | 1.61804  |     0.750525 |   nan        |
| Initial   | 0.770281 |     0.227528 |     0.141421 |

------------------------------------
##### Results for Inproved GA stoppimg rule
Runtime: 42.54 seconds, 7 steps, 6.08 seconds per step \
Optimal marginal Likelihood: -378.68, Estimated: -391.87, Initial -378.08
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.845078 |     0.568092 |    0.167374  |
| Girth     | 0.930362 |     0.554795 |    0.141421  |
| direct    | 0.401578 |     0.609516 |    0.0526782 |
| Initial   | 0.509902 |     0.364352 |    0.141421  |

------------------------------------
##### Results for pycma ga used
Runtime: 489.8 seconds, 16 steps, 30.61 seconds per step \
Optimal marginal Likelihood: -381.34, Estimated: -414.28, Initial -379.77
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.942127 |     0.505808 |     0.215139 |
| Girth     | 1.42144  |     0.632431 |   nan        |
| Initial   | 0.509902 |     0.489947 |     0.141421 |

------------------------------------
##### Results for pycma, higher sigma0
Runtime: 282.36 seconds, 10 steps, 28.24 seconds per step \
Optimal marginal Likelihood: -368.93, Estimated: -399.56, Initial -369.53
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.898735 |     0.48221  |     0.195257 |
| Girth     | 1.79316  |     0.515695 |   nan        |
| direct    | 0.458306 |     0.262779 |     0.139413 |
| Initial   | 0.509902 |     0.273981 |     0.141421 |

------------------------------------
##### Results for Monte Carlo with seed
Runtime: 97.6 seconds, 9 steps, 10.84 seconds per step \
Optimal marginal Likelihood: -376.79, Estimated: -399.07, Initial -372.28
|           |      rmse_A |   rmse_delta |   rmse_sigma |
|:----------|------------:|-------------:|-------------:|
| Estimated |    1.09615  |     0.970204 |     0.120425 |
| Girth     |    1.45372  |     0.34729  |     0.141421 |
| direct    | 2935.78     |  1035.68     |     0.014461 |
| Initial   |    0.509902 |     0.445143 |     0.141421 |

------------------------------------
##### Results for Monte Cralo with fixed sample per em-step
Runtime: 1.22 seconds, 5 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -381.06, Estimated: -373.26, Initial -377.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.500043 |     0.273166 |    0.0853068 |
| Girth     | 0.98542  |     0.334766 |    0.141421  |
| direct    | 0.57518  |     0.335156 |    0.0914744 |
| Initial   | 0.509902 |     0.431741 |    0.141421  |

------------------------------------
##### Results for Newton Raphson of Q_0 with approximate second derivative
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 2.2 seconds, 9 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -389.58, Estimated: -375.96, Initial -386.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.452731 |     0.428534 |     0.011738 |
| early_initial | 0.509902 |     0.451316 |     0.212132 |
| girth         | 1.54173  |     0.489318 |     0.164638 |
| early_direct  | 1.10778  |     0.416042 |     0.189961 |

------------------------------------
##### Results for Quasi Monte-Carlo with increasing sample-size based on ttest'scipy
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 589.08 seconds, 52 steps, 11.33 seconds per step \
Optimal marginal Likelihood: -371.52, Estimated: -366.22, Initial -373.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.287802 |     0.302635 |    0.0019799 |
| early_initial | 0.509902 |     0.292938 |    0.212132  |
| girth         | 1.1188   |     0.385494 |    0.0478773 |
| early_direct  | 0.282345 |     0.319402 |    0.0080443 |

------------------------------------
##### Results for Increasing Monte Carlo sample-size based on ttest with normal MC
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 3.3 seconds, 15 steps, 0.22 seconds per step \
Optimal marginal Likelihood: -371.46, Estimated: -365.03, Initial -374.57
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.660667 |     0.316008 |    0.0573464 |
| early_initial | 0.509902 |     0.28936  |    0.212132  |
| girth         | 1.0658   |     0.31617  |    0.12943   |
| early_direct  | 0.848957 |     0.59659  |    0.116858  |

## Procedure for better runtime

1. Add Performance Benchmark (done)
2. Add Vectorization to q_item (done)
3. Add better Initialization (done)
3. Add Vectorization to q_0 (done)
4. Add q_0 Derivation 1. + BFGS (done)
5. Add q_0 Derivation 2. + Newton Raphson
6. Use cython (maybe)

## Experiment 1: MIRT-2PL Parameter Recovery

In [10]:
from unittest import result

def mirt_param_recovery(sample_size, item_dimension = 20, latent_dimension=3, q_type="seperated", girth=True, stop_threshold=0.2, ensure_id=False, q_share=0.0) -> dict:
    
    #Simulate Responses
    simulation = item_response_simulation(item_dimension=item_dimension, latent_dimension=latent_dimension)
    parameter_dict = simulation.set_up(q_structure=q_type, q_share=q_share, ensure_id=ensure_id)
    sample = simulation.sample(sample_size=sample_size)

    #Define Population
    #population = respondent_population(latent_dimension=latent_dimension)
    real_latent_cov = parameter_dict["real_early_parameters"]["person_parameters"]["covariance"]
    print("Real latent covariance: {0}".format(real_latent_cov))
    

    #Sample responses
    #response_simulation_obj = response_simulation(population=population, item_dimension=item_dimension)
    #response_simulation_obj.initialize_random_q_structured_matrix(structure=q_type, ensure_id=ensure_id)
    #early_item_parameters = response_simulation_obj.initialize_random_item_parameters()
    parameter_dict["real_early_parameters"].update({"item_dimension": item_dimension, "latent_dimension": latent_dimension})
    real_early_parameters = parameter_dict["real_early_parameters"]#{"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=latent_dimension, item_dimension=item_dimension, Q=real_early_parameters["item_parameters"]["q_matrix"])
    print("Covariance matrix is good: {0}".format(model.check_sigma(real_latent_cov)))
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=100, stop_threshold=stop_threshold)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}}
    if girth==True:
        baselines["girth"] = {}

    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)

    return(performance_dict)

In [23]:
#Univariate IRT Recovery
result_dict = mirt_param_recovery(1000, q_type="singular", item_dimension=20, latent_dimension=1, stop_threshold=3)

Real latent covariance: [[1.]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 150
Step: 2: current parameter_diff: 22.742099580485075, current marginal loglikelihood: -8816.976449557947
EM Iteration 3
Current Monte Carlo Sample size: 165
Step: 3: current parameter_diff: 10.719114330591957, current marginal loglikelihood: -8747.334230330345
EM Iteration 4
Current Monte Carlo Sample size: 165
Step: 4: current parameter_diff: 4.741911411251834, current marginal loglikelihood: -8715.022616878563
EM Iteration 5
Current Monte Carlo Sample size: 165
Step: 5: current parameter_diff: 1.8142332751245616, current marginal loglikelihood: -8712.134520806754
EM Iteration 6
Current Monte Carlo Sample size: 181
Step: 6: current parameter_diff: 1.2343984015305836, current marginal loglikelihood: -8716.601796257768
EM Iteration 7
Current Monte Carlo Sample size: 199
Step: 7: current parameter_diff: 1.691136460449655, current marginal loglikelihood: -8705.167502668424
EM Iteration 8
Current Monte Carlo Sample size: 199
Step: 8: current parameter_di

In [25]:
print_result(result_dict, "Monte Carlo sample varying through ttest with normal MC")

------------------------------------
##### Results for Monte Carlo sample varying through ttest with normal MC
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 37.86 seconds, 20 steps, 1.89 seconds per step \
Optimal marginal Likelihood: -8714.19, Estimated: -8697.21, Initial -9512.15
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.182642 |     0.211132 |            0 |
| early_initial | 1.69549  |     1.49975  |            0 |
| girth         | 0.313235 |     4.2295   |            0 |


### Univariate IRT/Full GA

------------------------------------
##### Item Dimension=12, sample_size=200: Results for Fixed sample per em-step
Runtime: 4.38 seconds, 7 steps, 0.63 seconds per step \
Optimal marginal Likelihood: -1284.1, Estimated: -1273.09, Initial -1330.43
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.508687 |     0.196944 |            0 |
| Girth     | 0.377882 |     2.65875  |            0 |
| direct    | 0.305489 |     0.318168 |            0 |
| Initial   | 1.5583   |     0.861273 |            0 |

------------------------------------
##### Item Dimension=20, sample_size=1000: Results for Fixed sample per em-step
Runtime: 85.11 seconds, 19 steps, 4.48 seconds per step \
Optimal marginal Likelihood: -9199.01, Estimated: -9183.03, Initial -9784.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.352003 |     0.578688 |            0 |
| Girth     | 0.468473 |     4.98511  |            0 |
| Initial   | 1.66615  |     2.01114  |            0 |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 31.86 seconds, 10 steps, 3.19 seconds per step \
Optimal marginal Likelihood: -9619.61, Estimated: -9601.49, Initial -9970.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.175897 |     0.223543 |            0 |
| early_initial | 1.21508  |     1.78456  |            0 |
| girth         | 0.327131 |     4.71968  |            0 |

------------------------------------
##### Results for Monte Carlo sample varying through ttest with normal MC
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 37.86 seconds, 20 steps, 1.89 seconds per step \
Optimal marginal Likelihood: -8714.19, Estimated: -8697.21, Initial -9512.15
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.182642 |     0.211132 |            0 |
| early_initial | 1.69549  |     1.49975  |            0 |
| girth         | 0.313235 |     4.2295   |            0 |

In [12]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=2, q_type="seperated", girth=False, stop_threshold=1)

Real latent covariance: [[1.     0.1921]
 [0.1921 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 300
Step: 2: current parameter_diff: 9.087457477954748, current marginal loglikelihood: -12789.826209045792
EM Iteration 3
Current Monte Carlo Sample size: 300
Step: 3: current parameter_diff: 9.347286786290436, current marginal loglikelihood: -12431.792159340152
EM Iteration 4
Current Monte Carlo Sample size: 300
Step: 4: current parameter_diff: 5.994195183510891, current marginal loglikelihood: -12324.415808814525
EM Iteration 5
Current Monte Carlo Sample size: 330
Step: 5: current parameter_diff: 3.466965558217997, current marginal loglikelihood: -12291.666326556195
EM Iteration 6
Current Monte Carlo Sample size: 363
Step: 6: current parameter_diff: 3.0517259450824583, current marginal loglikelihood: -12274.293408739399
EM Iteration 7
Current Monte Carlo Sample size: 399
Step: 7: current parameter_diff: 1.6828800520798761, current marginal loglikelihood: -12268.44295499595
EM Iteration 8
Current Monte Carlo Sample size: 438
Step: 8: current parameter

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 13: current parameter_diff: 0.40565533319178126, current marginal loglikelihood: -12248.475980359144
EM Iteration 14
Current Monte Carlo Sample size: 581
Step: 14: current parameter_diff: 0.3981727860081429, current marginal loglikelihood: -12251.628743992456
EM Iteration 15
Current Monte Carlo Sample size: 639
Step: 15: current parameter_diff: 0.3759899463682548, current marginal loglikelihood: -12251.09622265484
EM Iteration 16
Current Monte Carlo Sample size: 702
Step: 16: current parameter_diff: 0.07730277535592268, current marginal loglikelihood: -12250.114423241039
EM Iteration 17
Current Monte Carlo Sample size: 772
Step: 17: current parameter_diff: 0.35397135818851444, current marginal loglikelihood: -12249.62312895641
Absolute diff in A:
[[ 0.14141196  0.        ]
 [ 0.14652275  0.        ]
 [ 0.24777834  0.        ]
 [ 0.39441549  0.        ]
 [ 0.12016762  0.        ]
 [ 0.32373102  0.        ]
 [ 2.04123377  0.        ]
 [ 0.07098307  0.        ]
 [ 0.06713308  0.    

In [13]:
print_result(performance_dict, "Fixed Issue with simulation Covariance")

------------------------------------
##### Results for Fixed Issue with simulation Covariance
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 51.07 seconds, 18 steps, 2.84 seconds per step \
Optimal marginal Likelihood: -11670.7, Estimated: -12247.32, Initial -13560.2
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.78631 |     0.262764 |    0.0418607 |
| early_initial |  2.54062 |     0.826317 |    0.217718  |


------------------------------------
##### Results for Seperated Q-Matrix
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 142.07 seconds, 26 steps, 5.46 seconds per step \
Optimal marginal Likelihood: -9384.38, Estimated: -10189.47, Initial -11459.9
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.00013 |      3.10678 |    0.0201383 |
| early_initial |  4.5183  |      2.5846  |    0.165491  |

------------------------------------
##### Results for BFGS for Q_0 with approximate Jacobi
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 61.55 seconds, 12 steps, 5.13 seconds per step \
Optimal marginal Likelihood: -9363.89, Estimated: -9711.58, Initial -10528.79
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.02507 |      1.83482 |    0.0645047 |
| early_initial |  2.06432 |      2.18337 |    0.289049  |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 325.73 seconds, 42 steps, 7.76 seconds per step \
Optimal marginal Likelihood: -9929.77, Estimated: -10179.96, Initial -10767.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.25777 |     0.965894 |   0.00557421 |
| early_initial |  1.98812 |     1.68516  |   0.719705   |

------------------------------------
##### Results for Fixed random Covariance 1, girth included
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 57.16 seconds, 14 steps, 4.08 seconds per step \
Optimal marginal Likelihood: -11668.66, Estimated: -11672.6, Initial -12307.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.183251 |    0.0861252 |    0.0215108 |
| early_initial | 0.786712 |    0.393667  |    0.0471788 |
| girth         | 1.28298  |    0.263374  |    0.350541  |

------------------------------------
##### Results for Fixed Issue with simulation Covariance
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 24.71 seconds, 11 steps, 2.25 seconds per step \
Optimal marginal Likelihood: -11186.38, Estimated: -11389.57, Initial -12010.77
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.83766 |      1.4664  |    0.0743169 |
| early_initial |  2.7041  |      2.66224 |    0.0627911 |

In [16]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=3, q_type="pyramid", girth=False, stop_threshold=10, ensure_id=True, q_share=0.6)

Real latent covariance: [[1.     0.5392 0.4093]
 [0.5392 1.     0.5406]
 [0.4093 0.5406 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))


[[1.         0.22255116 0.4851635 ]
 [0.22255116 1.         0.28710379]
 [0.4851635  0.28710379 1.        ]]
Step: 2: current parameter_diff: 22.836966115835242, current marginal loglikelihood: -11263.404209075177
EM Iteration 3


...

c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 877
[[ 1.         -0.18130942  0.49736388]
 [-0.18130942  1.         -0.04688012]
 [ 0.49736388 -0.04688012  1.        ]]
Step: 7: current parameter_diff: 4.21590743744123, current marginal loglikelihood: -10830.644137405307
EM Iteration 8


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 964
[[ 1.         -0.15152568  0.50240847]
 [-0.15152568  1.         -0.07665482]
 [ 0.50240847 -0.07665482  1.        ]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


In [8]:
print_result(performance_dict, "With Covariance Matrix partially aligned to Q-matrix")

NameError: name 'performance_dict' is not defined

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 200 \
Runtime: 121.22 seconds, 46 steps, 2.64 seconds per step \
Optimal marginal Likelihood: -2144.15, Estimated: -2155.08, Initial -2699.63
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  11.0034 |      1.77461 |     0.889828 |
| early_initial |  10.5116 |      2.15986 |     0.522199 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 313.22 seconds, 51 steps, 6.14 seconds per step \
Optimal marginal Likelihood: -9452.24, Estimated: -9813.87, Initial -11216.48
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.34115 |      1.4068  |     0.425982 |
| early_initial |  5.98238 |      2.25109 |     0.234288 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 30, sample size 2000 \
Runtime: 1521.51 seconds, 76 steps, 20.02 seconds per step \
Optimal marginal Likelihood: -30844.13, Estimated: -32521.34, Initial -37390.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.55959 |       1.9952 |    0.314331  |
| early_initial |  3.32034 |       3.514  |    0.0239562 |

------------------------------------
##### Results for BFGS for Q_0 with approximated gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 350.28 seconds, 64 steps, 5.47 seconds per step \
Optimal marginal Likelihood: -10498.87, Estimated: -11303.0, Initial -12740.14
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  10.2704 |     0.68597  |    0.318713  |
| early_initial |  10.4626 |     0.846172 |    0.0895367 |

------------------------------------
##### Results for BFGS for Q_0 with exact gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 152.91 seconds, 31 steps, 4.93 seconds per step \
Optimal marginal Likelihood: -11106.69, Estimated: -11211.51, Initial -12370.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.5074  |      2.54509 |     0.21123  |
| early_initial |  3.51819 |      2.66313 |     0.319647 |

------------------------------------
##### Results for Newton-Raphson for Q_0 with approximated second derivative
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 352.93 seconds, 62 steps, 5.69 seconds per step \
Optimal marginal Likelihood: -10237.22, Estimated: -10482.36, Initial -12569.49
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.23694 |      2.45127 |     0.508499 |
| early_initial |  5.36172 |      3.29141 |     0.640209 |

In [13]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=30, latent_dimension=3, q_type="full", girth=False, stop_threshold=10, q_share=0.0)

Real latent covariance: [[1.     0.     0.2545]
 [0.     1.     0.3721]
 [0.2545 0.3721 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 2: current parameter_diff: 41.855085339761395, current marginal loglikelihood: -12550.72345228223
EM Iteration 3
Current Monte Carlo Sample size: 660
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 3: current parameter_diff: 28.922785136715092, current marginal loglikelihood: -12131.893852917503
EM Iteration 4
Current Monte Carlo Sample size: 660
[[ 1.00000000e+000 -7.56483606e-249 -7.12199634e-243]
 [-7.56483606e-249  1.00000000e+000 -1.46994029e-260]
 [-7.12199634e-243 -1.46994029e-260  1.00000000e+000]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 4: current parameter_diff: 24.909956480277785, current marginal loglikelihood: -11869.365012663717
EM Iteration 5
Current Monte Carlo Sample size: 660
[[1.         0.07473132 0.19983603]
 [0.07473132 1.         0.24058055]
 [0.19983603 0.24058055 1.        ]]
Step: 5: current parameter_diff: 20.731814136925763, current marginal loglikelihood: -11739.105149945846
EM Iteration 6
Current Monte Carlo Sample size: 660
[[1.         0.02395303 0.25925938]
 [0.02395303 1.         0.26711545]
 [0.25925938 0.26711545 1.        ]]
Step: 6: current parameter_diff: 6.423400003338906, current marginal loglikelihood: -11720.71525563623
EM Iteration 7
Current Monte Carlo Sample size: 726
[[1.         0.02766565 0.2770886 ]
 [0.02766565 1.         0.30801604]
 [0.2770886  0.30801604 1.        ]]
Step: 7: current parameter_diff: 5.599935651052099, current marginal loglikelihood: -11726.261698793634
EM Iteration 8
Current Monte Carlo Sample size: 798
[[1.         0.03611896 0.29298871]
 [0.03611896

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


Step: 8: current parameter_diff: 8.88763105930743, current marginal loglikelihood: -11739.130271363902
EM Iteration 9
Current Monte Carlo Sample size: 877
[[1.         0.04450944 0.2860444 ]
 [0.04450944 1.         0.34325432]
 [0.2860444  0.34325432 1.        ]]
Step: 9: current parameter_diff: 6.972349751947515, current marginal loglikelihood: -11721.612152810394
EM Iteration 10
Current Monte Carlo Sample size: 964
[[1.         0.04267512 0.28987184]
 [0.04267512 1.         0.33339445]
 [0.28987184 0.33339445 1.        ]]
Step: 10: current parameter_diff: 13.349635834760017, current marginal loglikelihood: -11712.217814439908
EM Iteration 11
Current Monte Carlo Sample size: 1060
[[1.         0.05402791 0.27611641]
 [0.05402791 1.         0.32730623]
 [0.27611641 0.32730623 1.        ]]
Step: 11: current parameter_diff: 13.121900001706786, current marginal loglikelihood: -11752.478352914124
EM Iteration 12
Current Monte Carlo Sample size: 1166
[[1.         0.03472167 0.29289335]
 [0.0

KeyboardInterrupt: 

In [12]:
print_result(performance_dict, "With fixed Discrimination sampling and BFGS")

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 63.64 seconds, 14 steps, 4.55 seconds per step \
Optimal marginal Likelihood: -8684.81, Estimated: -8676.2, Initial -9809.83
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.933735 |     0.514423 |    0.0383988 |
| early_initial | 2.67536  |     0.636687 |    0.231996  |


------------------------------------
##### Results for With BFGS as covariance optimization Algorithm
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 160.7 seconds, 20 steps, 8.04 seconds per step \
Optimal marginal Likelihood: -11451.44, Estimated: -11746.37, Initial -12757.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.12413 |      1.19297 |     0.539132 |
| early_initial |  4.06187 |      1.22103 |     0.159225 |

------------------------------------
##### Results for With Cholesky covariance decomposition
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 60.82 seconds, 14 steps, 4.34 seconds per step \
Optimal marginal Likelihood: -11369.92, Estimated: -12102.54, Initial -13629.98
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.47435 |     0.400838 |     0.181127 |
| early_initial |  2.98819 |     0.686696 |     0.142997 |

------------------------------------
##### Results for With Cholesky covariance decomposition and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 94.77 seconds, 12 steps, 7.9 seconds per step \
Optimal marginal Likelihood: -15960.47, Estimated: -17344.15, Initial -19786.45
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  48.8803 |     0.777924 |     0.149531 |
| early_initial |  49.0831 |     1.33755  |     0.207159 |

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 91.68 seconds, 12 steps, 7.64 seconds per step \
Optimal marginal Likelihood: -12149.74, Estimated: -12157.89, Initial -14965.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  0.3929  |     0.249323 |    0.0275205 |
| early_initial |  2.20296 |     0.582838 |    0.380006  |

### Procedure for better model performance:

1. Implement MIRT Baseline (done)
2. Test GA with quadratic function (done)
3. use pycma (done)
4. Test mirt-functions (response-matrix-probability, done)
5. Read on MC MIRT (MC-Errorbounds usage or quasi-MC or importance sampling)
6. Use importance sampling
7. Implement Newton Raphson

### Estimeted Parameters

In [11]:
from scipy.optimize import approx_fprime

def func(C_vector):
    dim = np.sqrt(C_vector.shape).astype(np.int)[0]
    C = C_vector.reshape((dim, dim))
    sigma = np.matmul(C, C.transpose())
    return(sigma.flatten())

approx_fprime(f=func, xk=np.ones(2*2))

#Ich brauche wohl irgendeine Art von Matrixprodukt

array([[2., 2., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 2., 2.]])

In [12]:
approx_fprime(f=func, xk=np.arange(0,4
))

array([[1.49011612e-08, 2.00000001e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 3.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.00000000e+00, 3.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.00000000e+00, 6.00000000e+00]])

## Experiment 2: Competency Gain Prediction